In [1]:
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import xmltodict
import pandas as pd
import time
import os  

Set game and page you want

In [2]:
#enter game you want 
game_id = 1406

#start
start_game = game_id

#enter how many games to get?
number_games = 3

#enter starting page, probably 1
page_id = 1

In [5]:
#open up folder, probably get all game csv to append to
folder = r"C:\Users\peter\Desktop\tdi_capstone\bgg_api_data"

#read in master game csv
master_df = pd.read_csv(folder + "\\bgg_all_games.csv", index_col = 0)
master_df = master_df.reset_index(drop=True)

C:\Users\peter\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
master_df

,Username,Rating,Comment,Bggid
0,jackcres,10.0,NaN,1
1,FuManchu,10.0,Play this game. Play this game a lot.,1
2,HankM,10.0,hard to find enough players,1
3,jonnypolite,10.0,NaN,1
4,pwotruba,10.0,the older Moskito version is for 4 players onl...,1
...,...,...,...,...
2372464,Andramus,4.0,NaN,1405
2372465,alanmb,4.0,NaN,1405
2372466,JoeCthulhu,3.0,NaN,1405
2372467,Aglar,3.0,This game needs tweaking to approach the fun l...,1405


In [7]:
while game_id < (start_game + number_games):

    #get url
    url = f"https://boardgamegeek.com/xmlapi2/thing?id={game_id}&ratingcomments=1&page={page_id}&pagesize=100.xml"

    #get xml file
    #had issue try three times to ping site
    # https://stackoverflow.com/questions/15431044/can-i-set-max-retries-for-requests-request
    
    session = requests.Session()
    retry = Retry(connect=10, backoff_factor=0.9)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)

    response = session.get(url)
    
    data = xmltodict.parse(response.content)
    
    #IF bgg_id exist, go on, if not. reset page, add one to game, and move to next game, continue
    #if doesn't exist, 
    if "item" not in data['items']:
        #reset page id
        page_id = 1

        #go to next game
        game_id += 1
        
        time.sleep(6) # Sleep for 6 seconds
        
        #go to next while loop
        continue
    
    #if not a board game, move on
    if data['items']['item']['@type'] != 'boardgame':
        #reset page id
        page_id = 1

        #go to next game
        game_id += 1
        
        time.sleep(6) # Sleep for 6 seconds
        
        #go to next while loop
        continue
    
    #if no comments on game, move on
    if 'comments' not in data['items']['item'].keys():
        #reset page id
        page_id = 1

        #go to next game
        game_id += 1
        
        time.sleep(6) # Sleep for 6 seconds
        
        #go to next while loop
        continue
        
    
    #get game id
    bgg_id = int(data['items']['item']['@id'])
    
    master_dic_list = []

    #if there are ratings, get them, if not, stop
    while 'comment' in data['items']['item']['comments'].keys():

        #get list of comments
        dic_list = data['items']['item']['comments']['comment']

        #from xml, get list of dictionaries which contain user/ratings/comment
        dic_list = data['items']['item']['comments']['comment']

        master_dic_list.extend(dic_list)

        #then add 1 to page, ,pause 6 seconds, then get new page
        page_id += 1
        time.sleep(6) # Sleep for 6 seconds

        #get url
        url = f"https://boardgamegeek.com/xmlapi2/thing?id={game_id}&ratingcomments=1&page={page_id}&pagesize=100.xml"

        #get xml file
        response = requests.get(url)
        data = xmltodict.parse(response.content)


    #filter out any non dict, sometimes last comment is set as a string
    master_dic_list = [x for x in master_dic_list if isinstance(x, dict)]
    
    #if there are games with single comments they dont convert to list of dicts, dont want single comment games so deleted
    if not master_dic_list:
        #reset page id
        page_id = 1

        #go to next game
        game_id += 1
        
        time.sleep(6) # Sleep for 6 seconds
        
        #go to next while loop
        continue
        
    #make df from dict_list
    df = pd.DataFrame(master_dic_list)

    #rename columns
    df = df.rename(columns={"@username": "Username", "@rating": "Rating", "@value" : "Comment"})

    #convert rating from string to float
    df['Rating'] = df['Rating'].astype(float)

    #add the current game id to the dataframe
    df['Bggid'] = bgg_id

    #write game as individual file
    df.to_csv(folder + "\\" + str(bgg_id) + ".csv")

    #concat master dataframe with new dataframe
    master_df = pd.concat([master_df, df])

    #reset page id
    page_id = 1

    #go to next game
    game_id += 1

ConnectionError: ('Connection aborted.', TimeoutError(10060, 'A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond', None, 10060, None))

Save master csv

In [ ]:
master_df = master_df.reset_index(drop=True)

In [ ]:
#save master csv
master_df.to_csv(folder + "\\bgg_all_games.csv")

In [ ]:
master_df

Way to get all the csv into one df

In [ ]:
#open up folder, probably get all game csv to append to
folder = r"C:\Users\peter\Desktop\tdi_capstone\bgg_api_data"

#read in master game csv
master_df = pd.read_csv(folder + "\\1.csv", index_col = 0)
master_df = master_df.reset_index(drop=True)

In [ ]:
master_df

In [ ]:
for x in range(2,478):
    
    try:
        df = pd.read_csv(folder + "\\" + str(x) + ".csv", index_col = 0)
        df = df.reset_index(drop=True)
        master_df = pd.concat([master_df, df])
    except:    
        continue
    

In [ ]:
master_df = master_df.reset_index(drop=True)

In [ ]:
master_df

In [ ]:
#save master csv
master_df.to_csv(folder + "\\bgg_all_games_TEST.csv")

In [ ]:
url

In [ ]:
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry


session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

response = session.get(url)

In [ ]:
response1 = session.get(url)

In [ ]:
data_test = xmltodict.parse(response1.content)

In [ ]:
data_test

In [ ]:
#get url
url = f"https://boardgamegeek.com/xmlapi2/thing?id={game_id}&ratingcomments=1&page={page_id}&pagesize=100.xml"

#get xml file
response = requests.get(url)
data = xmltodict.parse(response.content)

In [ ]:
data

In [ ]:
data == data_test